In [10]:
"""
Solution for D2K Audubon project

Create images for bird detection
"""

import numpy as np
import utils.data_processing as dp
import utils.data_vis as vis
from utils.global_const import *

In [11]:
#######################################################################################
# Part 1 - Data processing

def test_get_file_names():
    ''' Test get_file_names() '''
    csv_files = dp.get_file_names(DATA_PATH + 'raw/', 'csv')
    jpg_files = dp.get_file_names(DATA_PATH + 'raw/', 'jpg')
    FILES['dataset'] = {'jpg': jpg_files, 'csv': csv_files}

test_get_file_names()

In [12]:
def test_csv_to_df():
    ''' Test csv_to_df() '''
    file_name = FILES['dataset']['csv'][0]
    data_df = dp.csv_to_df(file_name, COL_NAMES)
    # print(data_df)
    
test_csv_to_df()

In [13]:
def test_concat_frames():
    ''' Test concat_frames() ''' 
    concated_frame =  dp.concat_frames(FILES['dataset']['csv'], COL_NAMES)
    # print(concated_frame)

test_concat_frames()

In [14]:
def test_add_col():
    ''' Test add_col() '''
    values_dict = {}
    for key, vals in GROUPS.items():
        for val in vals:
            values_dict[val] = key
    frame = dp.concat_frames(FILES['dataset']['csv'], COL_NAMES)
    FRAMES['combined annotations'] = dp.add_col(frame, 'group_id', 'class_id', values_dict)
    # print(FRAMES['combined annotations'])

test_add_col()

In [15]:
def test_read_jpg():
    ''' test read_jpg() '''
    file_name = FILES['dataset']['jpg'][0]
    dp.read_jpg(file_name)

test_read_jpg()

In [16]:
#######################################################################################
# Part 2 - Data visualization

def test_plot_distribution():
    ''' Test plot_distribution() ''' 
    vis.plot_distribution(FRAMES['combined annotations'], "class_id", 
                          ("Frequency", "Bird Species", "Bird Species Distribution"), PLOTS_PATH, filt=100)
    vis.plot_distribution(FRAMES['combined annotations'], "group_id", 
                          ("Frequency", "Bird Group", "Bird Group Distribution"), PLOTS_PATH)
    vis.plot_distribution(FRAMES['combined annotations'].loc[FRAMES['combined annotations']['group_id'] == 'BRPE'], "class_id", 
                          ("Frequency", "Bird Species", "BRPE Bird Species Distribution"), PLOTS_PATH)
    vis.plot_distribution(FRAMES['combined annotations'].loc[FRAMES['combined annotations']['group_id'] == 'LGHT'], "class_id", 
                          ("Frequency", "Bird Species", "LGHT Bird Species Distribution"), PLOTS_PATH)            

# test_plot_distribution()

In [17]:
def test_plot_boxes():
    ''' Test plot_boxes() ''' 
    vis.plot_boxes(FILES['dataset']['jpg'][10], FILES['dataset']['csv'][10], 'Annonations', PLOTS_PATH)

# test_plot_boxes()

In [18]:
#######################################################################################
# Part 3 - Dataloader

# Split the dataset into trainset, testset, and valset
def test_split_img_annos():
    ''' Test split_img_annos() '''
    FILES['trainset'], FILES['testset'], FILES['valset'] = dp.split_img_annos(
        FILES['dataset']['jpg'], FILES['dataset']['csv'], (0.8, 0.1, 0.1), seed=2023)

test_split_img_annos()

In [19]:
# cropping
# Make a dataloader
# Train a model
# Evaluate the model

In [22]:
import torch

torch.ones((10,), dtype=torch.int64)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])